Импортируем библиотеки

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm

Загружаем данные

In [ ]:
images = np.load('data/images.npy')
labels = np.load('data/labels.npy')
images_sub = np.load('data/images_sub.npy')

Нормализуем данные

In [3]:
images = images.astype('float32') / 255.0
images_sub = images_sub.astype('float32') / 255.0

Делим на тренировочную и тестовую выборки

In [4]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

Преобразуем numpy в тензоры и переносим оси для pytorch

In [5]:
X_train = torch.from_numpy(X_train).permute(0, 3, 1, 2)
X_val = torch.from_numpy(X_val).permute(0, 3, 1, 2)
test_images = torch.from_numpy(images_sub).permute(0, 3, 1, 2)

CrossEntropyLoss требует целочисленных меток

In [6]:
y_train = torch.from_numpy(y_train).long()
y_val = torch.from_numpy(y_val).long()

Создаем тензоры из данных и меток классов

In [7]:
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

Создаем загрузчики

In [8]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_images, batch_size=64)

Задаем архитектуру модели

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 12 * 12, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 26),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

Создаем модель

In [10]:
device = torch.device('cuda')
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

Задаем функции обучения и валидации

In [11]:
def train(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
    
    return running_loss / len(loader), correct / total

def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
    
    return running_loss / len(loader), correct / total

Обучаем модель

In [ ]:
best_val_acc = 0.0

for epoch in range(30):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)
    
    print(f"Epoch {epoch+1}/{30}: "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    # Сохраняем лучшую модель по валидационной точности
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Сохранена новая лучшая модель с val_acc: {val_acc:.4f}")

100%|██████████| 250/250 [00:04<00:00, 52.00it/s]


Epoch 1/30: Train Loss: 3.1586 | Train Acc: 0.0971 | Val Loss: 2.2358 | Val Acc: 0.3675
Сохранена новая лучшая модель с val_acc: 0.3675


100%|██████████| 250/250 [00:03<00:00, 68.10it/s]


Epoch 2/30: Train Loss: 1.7501 | Train Acc: 0.4977 | Val Loss: 1.2289 | Val Acc: 0.6565
Сохранена новая лучшая модель с val_acc: 0.6565


100%|██████████| 250/250 [00:03<00:00, 68.27it/s]


...

Epoch 28/30: Train Loss: 0.0778 | Train Acc: 0.9749 | Val Loss: 0.5046 | Val Acc: 0.8702


100%|██████████| 250/250 [00:03<00:00, 68.52it/s]


Epoch 29/30: Train Loss: 0.0753 | Train Acc: 0.9754 | Val Loss: 0.4762 | Val Acc: 0.8830
Сохранена новая лучшая модель с val_acc: 0.8830


100%|██████████| 250/250 [00:03<00:00, 68.73it/s]


Epoch 30/30: Train Loss: 0.0767 | Train Acc: 0.9748 | Val Loss: 0.4727 | Val Acc: 0.8782


Загружаем лучшую модель

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

Делаем предсказания на тестовых данных

In [ ]:
predictions = []
with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        predictions.extend(predicted.cpu().numpy())

Сохраняем результаты

In [ ]:
submission = pd.DataFrame({
    'Id': range(len(predictions)),
    'Category': predictions
})
submission.to_csv('submission_final.csv', index=False)